In [1]:
from vespa.application import Vespa
from dotenv import load_dotenv  
load_dotenv()
import os

# Path to your security files
cert_path = os.getenv("VESPA_CERT")
key_path = os.getenv("VESPA_KEY")
endpoint_url = os.getenv("VESPA_ENDPOINT")

app = Vespa(
    url=endpoint_url,
    cert=cert_path,
    key=key_path
)

In [2]:
from typing import List
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def parse_results(results):
    parsed_results = []
    try:
        results = results.get_json()
        children = results.get('root', {}).get('children', [])
        
        if not children:
            logger.info("No results found in Vespa response")
            return parsed_results
        
        for result in children:
            # Safely extract fields with defaults
            fields = result.get('fields', {})
            matchfeatures = fields.get('matchfeatures', {})
            
            parsed_result = {
                'title': fields.get('title', '  '),
                'content': fields.get('content'),
                'page_number': fields.get('page_number'),
                'url': fields.get('url'),
                'tenant_id': fields.get('tenant_id'),
                'file_id': fields.get('file_id'),
                'relevance': result.get('relevance'),
                'cosine_scores': {
                    'cross_max_sim': matchfeatures.get('cross_max_sim'),
                    'semantic_summary': matchfeatures.get('semantic_question'),
                },
                'type': 'document'
            }
            parsed_results.append(parsed_result)
        
        return parsed_results
    except Exception as e:
        logger.error(f"Error parsing search results: {str(e)}")
        return parsed_results

def parse_summary_results(results):
    """Parse summary search results from Vespa"""
    parsed_results = []
    try:
        results = results.get_json()
        children = results.get('root', {}).get('children', [])
        
        if not children:
            logger.info("No summary results found in Vespa response")
            return parsed_results
        
        for result in children:
            # Safely extract fields with defaults
            fields = result.get('fields', {})
            matchfeatures = fields.get('matchfeatures', {})
            
            parsed_result = {
                'content_summary_chunk': fields.get('content_summary_chunk', ''),
                'url': fields.get('url', ''),
                'tenant_id': fields.get('tenant_id'),
                'file_id': fields.get('file_id'),
                'relevance': result.get('relevance', 0.0),
                'cosine_scores': {
                    'max_sim': matchfeatures.get('max_sim', 0.0)
                },
                'type': 'summary'
            }
            parsed_results.append(parsed_result)
        
        return parsed_results
    except Exception as e:
        logger.error(f"Error parsing summary search results: {str(e)}")
        return parsed_results

def filter_and_dedupe_results(results: List[dict], score_field: str, threshold: float = 0.75, unique_fields: List[str] = None) -> List[dict]:
    """Filter by score threshold and remove duplicates in one pass"""
    seen_combinations = set()
    filtered_unique = []
    
    for result in results:
        cosine_scores = result.get('cosine_scores', {})
        if cosine_scores.get(score_field, 0.0) < threshold:
            continue
            
        if unique_fields:
            identifier = tuple(result.get(field, '') for field in unique_fields)
            if identifier in seen_combinations:
                continue
            seen_combinations.add(identifier)
        
        filtered_unique.append(result)
    
    return filtered_unique

def query_pefund(query, source_param, filter_apollo=False):
    # Determine ranking profile based on source_param
    ranking_profile = "hybrid" if source_param == "summary" else "semantic-cross-sim"

    # Build the base YQL query
    base_conditions = (
        f"where tenant_id contains \"0f7a4b5f-a137-4a71-b518-1a04ba239b61\" "
        f"AND file_id contains \"9de6cf15-b315-4270-92c0-aa648b818949\" "
    )
    
    # Add apollo filter only if source_param is summary and filter_apollo is True
    if source_param == "summary" and filter_apollo:
        base_conditions += f"AND doc_type IN ('apollo') "
    
    initial_yql = (
        f"select * "
        f"from sources pefund.{source_param} "
        f"{base_conditions}"
        f"AND (userQuery() or ({{targetHits: 100}}nearestNeighbor(question_embeddings, q)))"
    )

    sbody = {
        'yql': initial_yql,
        'input.query(q)': f'embed(e5-small-v2, @query)',
        'input.query(qt)': f'embed(colbert, @query)',
        'query': query,
        'ranking.profile': ranking_profile,
        'targetHits': 20
    }

    return app.query(body=sbody)


In [3]:
from pydantic import BaseModel, Field
from typing import List, Dict, Optional
from langchain_google_genai import ChatGoogleGenerativeAI
class Source(BaseModel):
    url: str
    page_number: Optional[int] = None

class ChatResponse(BaseModel):
    response: str
    sources: List[Source]
    confidence_level: str

class QuestionList(BaseModel):
    questions: List[str] = Field(description="List of due diligence questions")

financial_chatbot_prompt = """
**Role**  
You are an expert Private-Equity / Fund-Management advisor with deep knowledge of due-diligence, portfolio management, capital markets, and valuation.

---

### Conversation context  
{previous_conversations}

### User’s current query  
{query}

### Retrieval results  
The list below mixes raw page snippets and summary chunks.  
Use them as your only factual source material.  
{search_results}
---
## Instructions
1. **Analyse the query in context** – decide if it is a follow-up or a new topic; keep the conversation coherent.

2. **Review the retrieval results**  
   • Extract only the information that answers the query.  
   • Cross-check multiple snippets to resolve contradictions.  
   • Note page numbers supplied in each result – you will need them for citations.

3. **Compose your answer**  
   • Write in professional, reader-friendly **Markdown**.  
   • Begin with a concise direct answer, then add any required detail, bullet points, tables, etc.  
   • Provide actionable insight or next steps if useful.

4. **Citations (MANDATORY)**  
   • Every factual statement that comes from the search results must end with an inline link of the form:  
     `[entity](url#page=page_number)`  
     - the entity part should be a number which is order or the doc in the search_results.
     Example: *Revenue grew 18 % YoY [citation text](https://fileserver/report.pdf#page=12)  
   • If multiple pages support the same line, list the links separated by commas.  
   • Do **not** invent page numbers or URLs.

5. **Knowledge boundaries**  
   • If the retrieval results do not contain enough information, say so explicitly and suggest what additional data would help.  
   • You may add general PE expertise **only** to clarify or interpret the retrieved facts—never as a substitute for missing data, and mark it clearly as “General PE insight”.

---

"""

# Set up a summarizer model pointing to Gemini (reusing existing key)
# We will use this in the summarize_chat function
gemini_summarizer = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    temperature=0.2, 
    api_key=os.getenv("GOOGLE_API_KEY")
)
structured_summarizer = gemini_summarizer.with_structured_output(ChatResponse)
structured_qa_model = gemini_summarizer.with_structured_output(QuestionList)

async def summarize_chat(query: str, documents: Dict, previous_conversations: List[Dict]) -> Dict:
    """
    Summarize the data for query using Gemini
    """
    summary_results = documents.get('summary_results', [])
    document_results = documents.get('document_results', [])
    
    doc_count = len(document_results) + len(summary_results)
    logger.info(f"Summarizing responses for - Query: '{query}', Documents: {doc_count}")

    try:
        structured_docs = []
        
        for i, doc in enumerate(document_results):
            structured_doc = {
                "title": doc.get('title', ''),
                "content": doc.get('content', ''),
                "page_number": doc.get('page_number', ''),
                "file_link": doc.get('url', '')
            }
            structured_docs.append(structured_doc)
            
        for i, doc in enumerate(summary_results):
            structured_doc = {
                "content_summary_chunk": doc.get('content_summary_chunk', ''),
                "file_link": doc.get('url', '')
            }
            structured_docs.append(structured_doc)
       
        documents_json = json.dumps(structured_docs, indent=2, ensure_ascii=False)
        
        # Construct prompt
        prompt = financial_chatbot_prompt.format(
            previous_conversations=str(previous_conversations),
            query=query,
            search_results=documents_json
        )
        
        # Call model
        # Note: structured_output ensures we get a Source object, bypassing the loose JSON markdown block request in the prompt text
        # but for compatibility with the user prompt, we assume the model handles the prompt instructions well.
        response_obj = await structured_summarizer.ainvoke(prompt)
        
        logger.info(f"Successfully summarized responses for query: {query}")
        # Return as dict
        return response_obj.model_dump()
        
    except Exception as e:
        logger.error(f"Error summarizing with Gemini: {str(e)} - Query: '{query}'")
        return {"response": "Error generating summary.", "sources": [], "confidence_level": "Low", "error": str(e)}


In [4]:
PE_DUE_DILIGENCE_PROMPT = """
[SYSTEM OPERATING DIRECTIVE: ACT AS A SENIOR MANAGING DIRECTOR & HEAD OF DUE DILIGENCE]
You are conducting a final Investment Committee (IC) review for a multi-billion dollar acquisition. Your tone is skeptical, intellectually aggressive, and focused on protecting Limited Partner (LP) capital.

**TARGET SEGMENT:** {header_title}
**FINANCIAL DATA SOURCE:** ---
{chunk_content}
---

**CORE OBJECTIVE:**
Generate 5-7 "Surgical Strike" questions. Your goal is to dismantle the management's narrative and uncover the "Truth behind the GAAP/IFRS numbers." 

**STRATEGIC QUESTIONING PILLARS:**
1. **QoE & Accrual Integrity:** Target non-cash padding or "one-time" P&L benefits. Look for provision releases in Zakat/Tax (Note 31) or gains from Discontinued Operations (Note 34) masking operational decay.
2. **Capital Intensity & Bow Waves:** Cross-reference PP&E against Depreciation. Question if "Repairs and Maintenance" spend is sufficient to prevent asset obsolescence or if a massive CAPEX "bow wave" is deferred.
3. **Liquidity & Shadow Financing:** Analyze the gap between Current Liabilities and Cash on Hand. Specifically target the reliance on "Trade Payables" and "Other Current Liabilities" as a primary, unsecured financing vehicle.
4. **Macro/FX Sensitivity:** Focus on the SAR/USD peg. Target "Exchange difference on translation" losses and demand a stress-tested LTM Debt-to-Equity ratio under a 5-10% de-pegging scenario.
5. **Structural Leakage:** Analyze "Non-Controlling Interests" (NCI) and "Associates." Identify why capital is leaking to minority partners (NCI dividends) while parent Retained Earnings are under pressure.

**MANDATORY CONSTRAINTS:**
- **BREVITY:** Max 2-3 sentences per question. No preamble. No "Considering the fact that..." 
- **DATA-DENSITY:** You MUST cite specific SAR values and year-over-year (YoY) changes from the data.
- **ADVANCED METRICS:** Frame questions around ROIC, FCF Conversion, Working Capital Intensity, and LTM Bridge Analysis.
- **SKEPTICISM:** Assume management is hiding under-performance in "Other Assets" or "Other Reserves."
- NO generic questions (e.g., "Why did revenue change?").
- USE advanced financial metrics: ROIC, Free Cash Flow Conversion, Working Capital Intensity, and LTM Bridge Analysis.
- DO NOT be limited by the provided text; if the text suggests a risk (like a discontinued operation), ask about the strategic fallout even if the answer isn't in the chunk.
"""

In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter
import time
import json
import os

SEARCH_THRESHOLD = 0.5

async def run_sequential_pipeline():
    with open("input/sample_table.md", "r", encoding="utf-8") as f:
        md_content = f.read()

    headers_to_split_on = [("###", "Header_Title")]
    splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    sections = splitter.split_text(md_content)

    final_report_pefund = []
    final_report_summary = []
    final_report_summary_apollo = []
    
    print(f"[INFO] Found {len(sections)} financial sections. Starting sequential processing...")

    for section in sections:
        header = section.metadata.get("Header_Title", "Financial Section")
        print(f"\n{'='*60}\nSECTION: {header}\n{'='*60}")

        section_results_pefund = {
            "section_title": header,
            "timestamp": time.ctime(),
            "due_diligence_inquiries": []
        }
        section_results_summary = {
            "section_title": header,
            "timestamp": time.ctime(),
            "due_diligence_inquiries": []
        }
        section_results_summary_apollo = {
            "section_title": header,
            "timestamp": time.ctime(),
            "due_diligence_inquiries": []
        }

        formatted_prompt = PE_DUE_DILIGENCE_PROMPT.format(
            header_title=header,
            chunk_content=section.page_content
        )

        try:
            response = structured_qa_model.invoke(formatted_prompt)
            questions = response.questions
            print(f"[LLM] Generated {len(questions)} high-stakes questions.")
        except Exception as e:
            print(f"[LLM ERROR] Failed to generate questions: {e}")
            continue

        for i, q in enumerate(questions, 1):
            print(f"\n[QUESTION {i}]: {q}")

            # 1. Fetch from PEFUND
            filtered_pefund = []
            try:
                print("  -> Querying pefund...")
                raw = query_pefund(q, "pefund")
                parsed = parse_results(raw)
                filtered_pefund = filter_and_dedupe_results(parsed, 'cross_max_sim', SEARCH_THRESHOLD, ['file_id', 'page_number'])
            except Exception as e:
                print(f"  [ERROR] pefund: {e}")

            # 2. Fetch from SUMMARY (without apollo filter)
            filtered_summary = []
            try:
                print("  -> Querying summary...")
                raw = query_pefund(q, "summary", filter_apollo=False)
                parsed = parse_summary_results(raw)
                filtered_summary = filter_and_dedupe_results(parsed, 'max_sim', SEARCH_THRESHOLD, ['file_id', 'content_summary_chunk'])
            except Exception as e:
                print(f"  [ERROR] summary: {e}")

            # 3. Fetch from SUMMARY with apollo filter
            filtered_summary_apollo = []
            try:
                print("  -> Querying summary (apollo)...")
                raw = query_pefund(q, "summary", filter_apollo=True)
                parsed = parse_summary_results(raw)
                filtered_summary_apollo = filter_and_dedupe_results(parsed, 'max_sim', SEARCH_THRESHOLD, ['file_id', 'content_summary_chunk'])
            except Exception as e:
                print(f"  [ERROR] summary apollo: {e}")

            # 4. Generate THREE SEPARATE summaries - one for each source
            
            # Summary for PEFUND only
            rag_response_pefund = {}
            try:
                docs_aggregation_pefund = {
                    "document_results": filtered_pefund,
                    "summary_results": []
                }
                print("  -> Generating RAG summary (pefund)...")
                rag_response_pefund = await summarize_chat(q, docs_aggregation_pefund, [])
                print("  -> Summary generated (pefund).")
            except Exception as e:
                print(f"  [ERROR] summarization pefund: {e}")

            # Summary for SUMMARY (non-apollo) only
            rag_response_summary = {}
            try:
                docs_aggregation_summary = {
                    "document_results": [],
                    "summary_results": filtered_summary
                }
                print("  -> Generating RAG summary (summary)...")
                rag_response_summary = await summarize_chat(q, docs_aggregation_summary, [])
                print("  -> Summary generated (summary).")
            except Exception as e:
                print(f"  [ERROR] summarization summary: {e}")

            # Summary for SUMMARY APOLLO only
            rag_response_apollo = {}
            try:
                docs_aggregation_apollo = {
                    "document_results": [],
                    "summary_results": filtered_summary_apollo
                }
                print("  -> Generating RAG summary (apollo)...")
                rag_response_apollo = await summarize_chat(q, docs_aggregation_apollo, [])
                print("  -> Summary generated (apollo).")
            except Exception as e:
                print(f"  [ERROR] summarization apollo: {e}")

            # Append to separate report lists with their respective RAG summaries
            section_results_pefund["due_diligence_inquiries"].append({
                "question": q,
                "rag_summary": rag_response_pefund
            })
            
            section_results_summary["due_diligence_inquiries"].append({
                "question": q,
                "rag_summary": rag_response_summary
            })
            
            section_results_summary_apollo["due_diligence_inquiries"].append({
                "question": q,
                "rag_summary": rag_response_apollo
            })

        final_report_pefund.append(section_results_pefund)
        final_report_summary.append(section_results_summary)
        final_report_summary_apollo.append(section_results_summary_apollo)

    # Save all three reports
    with open("outputs/output_md_files/pe_due_diligence_results_pefund.json", "w", encoding="utf-8") as f:
        json.dump(final_report_pefund, f, indent=4, ensure_ascii=False)
    print(f"\n[SUCCESS] Pefund report saved to outputs/output_md_files/pe_due_diligence_results_pefund.json")

    with open("outputs/output_md_files/pe_due_diligence_results_summary.json", "w", encoding="utf-8") as f:
        json.dump(final_report_summary, f, indent=4, ensure_ascii=False)
    print(f"[SUCCESS] Summary report saved to outputs/output_md_files/pe_due_diligence_results_summary.json")

    with open("outputs/output_md_files/pe_due_diligence_results_summary_apollo.json", "w", encoding="utf-8") as f:
        json.dump(final_report_summary_apollo, f, indent=4, ensure_ascii=False)
    print(f"[SUCCESS] Summary Apollo report saved to outputs/output_md_files/pe_due_diligence_results_summary_apollo.json")

# Run the async pipeline
await run_sequential_pipeline()

INFO:google_genai.models:AFC is enabled with max remote calls: 10.


[INFO] Found 1 financial sections. Starting sequential processing...

SECTION: CONSOLIDATED STATEMENT OF FINANCIAL POSITION


INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"


[LLM] Generated 6 high-stakes questions.

[QUESTION 1]: The dramatic SAR 1,038,714 reduction in Current Provisions and SAR 1,296,379 reduction in Zakat Payable in 2024 raises red flags. Detail precisely how much of the SAR 2,335,093 combined reduction was a P&L benefit, rather than genuine cash-settled obligations, and reconcile this against the SAR 8,059,563 decline in Retained Earnings.
  -> Querying pefund...
  -> Querying summary...
  -> Querying summary (apollo)...


INFO:__main__:No summary results found in Vespa response
INFO:__main__:Summarizing responses for - Query: 'The dramatic SAR 1,038,714 reduction in Current Provisions and SAR 1,296,379 reduction in Zakat Payable in 2024 raises red flags. Detail precisely how much of the SAR 2,335,093 combined reduction was a P&L benefit, rather than genuine cash-settled obligations, and reconcile this against the SAR 8,059,563 decline in Retained Earnings.', Documents: 10
INFO:google_genai.models:AFC is enabled with max remote calls: 10.


...

  -> Summary generated (pefund).
  -> Generating RAG summary (summary)...


INFO:__main__:Successfully summarized responses for query: Parent Retained Earnings declined by SAR 8,059,563 while Non-Controlling Interests decreased by only SAR 768,900 and Investments in Associates declined by SAR 5,801,516. Explain this disproportionate capital leakage from the parent, quantify the NCI dividend outflow, and provide an LTM ROIC calculation for the core operating assets, demonstrating how value is being generated for LP capital holders, not just minority partners.
INFO:__main__:Summarizing responses for - Query: 'Parent Retained Earnings declined by SAR 8,059,563 while Non-Controlling Interests decreased by only SAR 768,900 and Investments in Associates declined by SAR 5,801,516. Explain this disproportionate capital leakage from the parent, quantify the NCI dividend outflow, and provide an LTM ROIC calculation for the core operating assets, demonstrating how value is being generated for LP capital holders, not just minority partners.', Documents: 0
INFO:google_gena

  -> Summary generated (summary).
  -> Generating RAG summary (apollo)...


INFO:__main__:Successfully summarized responses for query: Parent Retained Earnings declined by SAR 8,059,563 while Non-Controlling Interests decreased by only SAR 768,900 and Investments in Associates declined by SAR 5,801,516. Explain this disproportionate capital leakage from the parent, quantify the NCI dividend outflow, and provide an LTM ROIC calculation for the core operating assets, demonstrating how value is being generated for LP capital holders, not just minority partners.


  -> Summary generated (apollo).

[SUCCESS] Pefund report saved to outputs/output_md_files/pe_due_diligence_results_pefund.json
[SUCCESS] Summary report saved to outputs/output_md_files/pe_due_diligence_results_summary.json
[SUCCESS] Summary Apollo report saved to outputs/output_md_files/pe_due_diligence_results_summary_apollo.json
